In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=5be02a2b0c1c031224a53c0bc2729c0ecedc07aa09108f6c97864c00d9e78152
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.appName('Movie Lens Recommendation').getOrCreate()

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines = spark.read.text("/content/ratings.dat").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=int(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions.show()

import math
result = predictions.rdd.map(lambda row: row['prediction'] - row['rating']).map(lambda x: x*x).filter(lambda x: not math.isnan(x))
mse = result.reduce(lambda x,y: x+y)


+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|      1|     5|978824268| 4.2697434|
|     1|    150|     5|978301777| 4.2068405|
|     1|    588|     4|978824268| 4.2679925|
|     1|    720|     3|978300760| 3.7368367|
|     1|    919|     4|978301368|  4.763472|
|     1|   1035|     5|978301753|  4.566914|
|     1|   1287|     5|978302039| 3.9222572|
|     1|   1566|     4|978824330|   3.41122|
|     1|   2294|     4|978824291| 4.0217667|
|     1|   3105|     5|978301713| 4.2682047|
|     1|   3408|     4|978300275|  3.988714|
|     2|     21|     1|978299839|  3.501818|
|     2|    235|     3|978299351| 2.4792988|
|     2|    480|     5|978299809| 3.8199909|
|     2|    589|     4|978299773| 3.7223487|
|     2|    920|     5|978298775| 4.2391424|
|     2|    982|     4|978299269| 3.3213933|
|     2|   1193|     5|978298413| 4.3011193|
|     2|   1198|     4|978298124| 4.3633194|
|     2|  